<a href="https://colab.research.google.com/github/CC4mpbe11/PenguinCategorization/blob/main/PenguinCategorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow-gpu

In [ ]:
pip install -q sklearn

In [ ]:
pip install -q keras

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from six.moves import urllib
from IPython.display import clear_output
from google.colab import files
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from pathlib import Path  

import io
import tensorflow as tf
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100
import tensorflow.compat.v2.feature_column as fc

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load

penguins_path = Path('/content/drive/MyDrive/Datasets/Penguin Data/penguins.csv')
penguins = pd.read_csv(penguins_path)

In [ ]:
# Save

penguins_path = Path('/content/drive/MyDrive/Datasets/Penguin Data/penguins.csv')
penguins_path.parent.mkdir(parents=True, exist_ok=True)
penguins.to_csv(penguins_path, index=False)

In [ ]:
penguins['sex'] = penguins['sex'].replace("Male", 1)
penguins['sex'] = penguins['sex'].replace("Female", 0)

In [ ]:
penguins['species'] = penguins['species'].replace("Adelie", 0)
penguins['species'] = penguins['species'].replace("Gentoo", 1)
penguins['species'] = penguins['species'].replace("Chinstrap", 2)

In [ ]:
x = penguins[['island',
          'bill_length_mm',
          'bill_depth_mm',
          'flipper_length_mm',
          'body_mass_g',
          'sex']]

In [ ]:
y = penguins[['species']]

In [ ]:
encoded_x = pd.get_dummies(x['island'], prefix='island')

In [ ]:
encoded_x['bill_length_mm'] = x['bill_length_mm']
encoded_x['bill_depth_mm'] = x['bill_depth_mm']
encoded_x['flipper_length_mm'] = x['flipper_length_mm']
encoded_x['body_mass_g'] = x['body_mass_g']
encoded_x['sex'] = x['sex']

In [ ]:
y_train, y_test, x_train, x_test = train_test_split(y, encoded_x, test_size=0.1)

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [ ]:
penguins_feature_columns = []
for key in encoded_x.keys():
  penguins_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=penguins_feature_columns,
    hidden_units=[30,10],
    n_classes=3)

classifier.train(
    input_fn = lambda: input_fn(x_train, y_train, training=True),
    steps = 5000)

In [ ]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(x_test, y_test, training=False))
print(eval_result)
print('\nTest set accuracy: {accuracy:0.3}\n'.format(**eval_result))